In [1]:
%load_ext autoreload
%autoreload 2

# import IPython
# IPython.get_ipython().run_line_magic('cd', '..')  # Go up one directory

from buffer import AllActivationBuffer
from trainers.top_k import TrainerSCAE, AutoEncoderTopK
from training import trainSCAE

from datasets import load_dataset
import torch as t
from nnsight import LanguageModel

#
device = "cuda:0" if t.cuda.is_available() else "cpu"
model = LanguageModel("gpt2", device_map=device)
model.eval()


dataset = load_dataset(
    'Skylion007/openwebtext', 
    split='train', 
    streaming=True,
    trust_remote_code=True
    )

class CustomData():
    def __init__(self, dataset):
        self.data = iter(dataset)

    def __iter__(self):
        return self

    def __next__(self):
        return next(self.data)['text']

data = CustomData(dataset)

In [2]:
C = 10
expansion = 8
k = 64 # TODO

num_features = model.config.n_embd * expansion

In [3]:
n_layer = model.config.n_layer

submodules = {}
for layer in range(n_layer):
    submodules[f"mlp_{layer}"] = (model.transformer.h[layer].mlp, "in_and_out")
    submodules[f"attn_{layer}"] = (model.transformer.h[layer].attn, "out")


buffer = AllActivationBuffer(
    data=data,
    model=model,
    submodules=submodules,
    d_submodule=model.config.n_embd, # output dimension of the model component
    n_ctxs=128,  # you can set this higher or lower depending on your available memory
    device="cuda",
    out_batch_size = 1024,
    refresh_batch_size = 256,
) 

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [4]:
important_features = {f"mlp_{layer}": t.randint(0, num_features, (num_features, C))
                        for layer in range(n_layer)}

# Get submodule names from the submodules dictionary
submodule_names = list(submodules.keys())

In [9]:
# TEST
trainer_cfg = {
    "trainer": TrainerSCAE,
    "activation_dims": {name: model.config.n_embd for name in submodule_names},
    "dict_sizes": {name: model.config.n_embd * expansion for name in submodule_names},
    "ks": {name: k for name in submodule_names},
    "device": buffer.device,
    "submodules": submodules,
    "important_features": important_features,
}

trainer_class = trainer_cfg.pop("trainer")
trainer = trainer_class(**trainer_cfg)

inputs, targets = next(buffer)
trainer.update(0, inputs, targets)

299476.4375

In [ ]:
trainer_cfg = {
    "trainer": TrainerSCAE,
    "activation_dims": {name: model.config.n_embd for name in submodule_names},
    "dict_sizes": {name: model.config.n_embd * expansion for name in submodule_names},
    "ks": {name: k for name in submodule_names},
    "device": buffer.device,
    "submodules": submodules,
    "important_features": important_features,
}

# Run the training
trainer = trainSCAE(
    buffer=buffer,
    trainer_cfg=trainer_cfg,
    steps=30000,
    save_steps=1000,
    save_dir="sae_checkpoints",
    log_steps=100,
    use_wandb=False, 
    use_sparse_connections=True,
)

#aa9a791c5e40fa7ab2f08d555ff72352c1cecaa2

Loss: 336464.3750:   0%|          | 10/30000 [00:08<7:16:34,  1.14it/s]


KeyboardInterrupt: 